In [39]:
import whisper
import os
import re
import csv
import pandas as pd
import jiwer
import torch
import tiktoken
import matplotlib
import seaborn
from transformers import pipeline
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

## Step 1: Speech-to-Text (ใช้ Whisper)

In [ ]:
MODEL_NAME = "biodatlab/whisper-th-medium-combined"
device = 0 if torch.cuda.is_available() else "cpu"
pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)

Device set to use cpu


In [41]:
audio_file = "/Users/momo/Downloads/Telesales.wav"
transcription = pipe(
    audio_file,
    generate_kwargs={"language": "<|th|>", "return_timestamps": False}, 
)["text"]

/Users/momo/Library/Python/3.9/lib/python/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [ ]:
#Save Transcription
with open("transcription1.txt", "w", encoding="utf-8") as txt_file:
    txt_file.write(transcription)

In [ ]:
# #Save as CSV
# csv_filename = "transcription1.csv"
# with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
#     writer = csv.writer(file)
#     writer.writerow(["Transcription"])
#     writer.writerow([transcription])

## Preprocessing

In [ ]:
import jiwer
from pythainlp.spell import correct

with open("transcription.txt", "r", encoding="utf-8") as file:
    transcription = file.read().strip()

#แ
transcription_corrected = " ".join([correct(word) for word in transcription.split()])

with open("transcription_cleaned.txt", "w", encoding="utf-8") as file:
    file.write(transcription_corrected)

## Step 2: Evaluate Speech Recognition (WER & CER)

In [ ]:
# อ่าน ground_truth จากไฟล์ transcription.txt
with open("transcription.txt", "r", encoding="utf-8") as file:
    ground_truth = file.read().strip()  # ลบช่องว่าง

with open("text.txt", "r", encoding="utf-8") as file:
    predicted_text = file.read().strip()  # ลบช่องว่าง

def evaluate_transcription(ground_truth, predicted_text):
    wer = jiwer.wer(ground_truth, predicted_text)
    cer = jiwer.cer(ground_truth, predicted_text)
    return {"WER": wer, "CER": cer}

In [ ]:
with open("transcription.txt", "r", encoding="utf-8") as file:
    ground_truth = file.read().strip()  # ลบช่องว่าง

with open("text.txt", "r", encoding="utf-8") as file:
    predicted_text = file.read().strip()  # ลบช่องว่าง

def evaluate_transcription(ground_truth, predicted_text):
    wer = jiwer.wer(ground_truth, predicted_text)
    cer = jiwer.cer(ground_truth, predicted_text)
    return {"WER": wer, "CER": cer}

In [98]:
# ประเมินคุณภาพของ Speech Recognition
eval_result = evaluate_transcription(ground_truth, transcription)
print("Speech Recognition Evaluation:", eval_result)

Speech Recognition Evaluation: {'WER': 0.109375, 'CER': 0.004794520547945206}


## Step 3: Chunking

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    
    chunk_overlap=200,  
    length_function=len,
)

In [47]:
chunks = text_splitter.split_text(transcription)
documents = [Document(page_content=chunk) for chunk in chunks]

In [48]:
len(documents)

4

In [ ]:
persist_directory = 'chroma_db/'

vectordb2 = Chroma(embedding_function=embedding_model,
                 persist_directory=persist_directory)

In [ ]:
vectorstore._collection.count()

8

In [ ]:
# # load from disk
# persist_directory = 'chroma_db/'
# temp_db = Chroma(collection_name="langchain",
#                  persist_directory=persist_directory, 
#              embedding_function=HuggingFaceEmbeddings)

In [ ]:
# temp_db._collection.count()

0

## Step 5: LLM for Sales Coaching & Feature Extraction (ใช้ Ollama)

In [ ]:
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

llm = Ollama(model="mistral")  #"mixtral"

retrieval_prompt = """
You are a professional sales trainer specializing in insurance sales coaching.  
- Your task is to provide **clear, structured, and actionable** advice based on only the retrieved documents. 
<Response Guidelines>
- Use three sentences maximum and keep the answer concise
- You MUST NOT mention something like "according to the document" or "context" in the answer.
- You MUST answer in English if the question contains all English. You MUST answer in Thai if the question contains Thai.
</Response Guidelines>
<Query>
{question}
</Query>
<Context>
{context}
</Context>
<Answer>
"""

prompt_template = ChatPromptTemplate.from_template(retrieval_prompt)

In [78]:
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt_template
    | llm
    | StrOutputParser()
)

In [79]:
rag_chain_with_source = RunnableParallel(
    {
        "context": lambda query: vectorstore.similarity_search(query, k=5),
        "question": RunnablePassthrough()
    }
).assign(answer=rag_chain_from_docs)

## Step 6: Feature Extraction Evaluation

In [ ]:
def evaluate_feature_extraction(transcription, extracted_features):
    relevant_keywords = ["เงินคืน", "รับประกัน", "การันตี"]
    extracted_count = sum(1 for word in relevant_keywords if word in extracted_features.lower())
    return {"Relevant Features Found": extracted_count, "Total Features Expected": len(relevant_keywords)}

extracted_features = "เงินคืนการันตีทุกปี 24 ปี"
feature_eval = evaluate_feature_extraction(transcription, extracted_features)
print("Feature Extraction Evaluation:", feature_eval)

Feature Extraction Evaluation: {'Relevant Features Found': 2, 'Total Features Expected': 3}


## Step 7: Sales Coaching Evaluation

In [ ]:
def evaluate_sales_coaching(question, expected_response):
    response = llm.invoke(question)
    similarity = jiwer.wer(expected_response, response)
    return {"WER Similarity Score": similarity, "LLM Response": response}

question = "จะเชิญชวนให้ลูกค้าซื้อกรมธรรมได้อย่างไร?"
expected_response = "จะเชิญชวนโดยการเน้นถึงผลประโยชน์และความคุ้มครอง"
sales_eval = evaluate_sales_coaching(question, expected_response)
print("Sales Coaching Evaluation:", sales_eval)

KeyboardInterrupt: 

In [ ]:
# Step 7: Sales Coaching Evaluation

def evaluate_sales_coaching(question, expected_responses):
    response = llm.invoke(question)
    
    transform = jiwer.Compose([
        jiwer.ToLowerCase(),
        jiwer.RemovePunctuation(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip()
    ])
    
    llm_response_clean = transform(response)
    wer_scores = [jiwer.wer(transform(resp), llm_response_clean) for resp in expected_responses]
    
    return {"WER Similarity Score": min(wer_scores), "LLM Response": response}


In [ ]:
question = "จะเชิญชวนให้ลูกค้าซื้อกรมธรรม์ได้อย่างไร?"
expected_responses = [
    "คุณสามารถเน้นถึงประโยชน์ของกรมธรรม์ เช่น ความคุ้มครอง การลดหย่อนภาษี และผลตอบแทนที่มั่นคง",
    "คุณควรอธิบายถึงผลประโยชน์ของกรมธรรม์ที่เหมาะกับลูกค้า",
    "การขายกรมธรรม์ต้องเน้นความคุ้มค่าและผลตอบแทน"
]

In [ ]:

sales_eval = evaluate_sales_coaching(question, expected_responses)
print("Sales Coaching Evaluation:", sales_eval)

Sales Coaching Evaluation: {'WER Similarity Score': 8.4, 'LLM Response': 'สำหรับการขายกรมธรรม์, คุณสามารถใช้วิธีการต่อไปนี้ในการชวนลูกค้าซื้อ:\n\n1. แสดงภาพและวิธีการใช้งานของกรมธรรม์เล็กน้อยๆ ที่หลับหวาด (Demo video)\n2. สรุปผลตอบแหล่งข้อมูลที่นำไปใช้บำรุงความเชื่อถือ (Customer testimonials)\n3. สนับสนุนการซื้อด้วยบัญชีโยงตามธุรกิจ (Business financing)\n4. แสดงสภาพแวดล้อมที่สะอาดและเป็นสถานที่ทำงานปลอดภัย (Clean and safe working environment)\n5. ใช้กรณีผลตอบของการใช้งานกรมธรรม์เพื่อสอนลูกค้าได้รู้จัก (Case studies of the benefits of using the product)\n6. แสดงภาพและวิธีที่ผู้ใช้สามารถเป็นสามัญ (Easy to use)\n7. ให้โปรโมชันแบ่งปฏิทิน (Promotions throughout the year)\n8. และอื่นๆ ๆ\n\nขอใจว่าคำแนะนำนี้ได้เห็นสมคุณภาพกับท่านและใช้งานได้เรื่อยๆ ในการขายกรมธรรม์ได้อย่างดีในภาพถูกต้อง!'}
